# HyperclassifierSearch Examples

# Example 1: train multiple classifiers

In [1]:
# import of the package (after e.g. 'pip install HyperclassifierSearch')
from HyperclassifierSearch import HyperclassifierSearch

# usage dependent imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# example dataset
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

In [2]:
# define classifiers and parameters for hyperfitting
models = {
    'LogisticRegression': LogisticRegression(solver='lbfgs', max_iter=10000),
    'RandomForestClassifier': RandomForestClassifier()
}
params = { 
    'LogisticRegression': { 'C': [0.1, 1, 2] },
    'RandomForestClassifier': { 'n_estimators': [16, 32] }
}

# run search
X_train, X_test, y_train, y_test = train_test_split(X, y)
search = HyperclassifierSearch(models, params)
best_model = search.train_model(X, y, cv=2, iid=False)
search.evaluate_model()

Search for LogisticRegression ...
Search for RandomForestClassifier ...
Search is done.
results round for: LogisticRegression
results round for: RandomForestClassifier


,Estimator,params,split0_test_score,split1_test_score,mean_test_score,std_test_score
0,RandomForestClassifier,{'n_estimators': 32},0.943860,0.954225,0.949043,0.005183
1,LogisticRegression,{'C': 2},0.947368,0.943662,0.945515,0.001853
2,LogisticRegression,{'C': 1},0.943860,0.943662,0.943761,0.000099
3,LogisticRegression,{'C': 0.1},0.943860,0.936620,0.940240,0.003620
4,RandomForestClassifier,{'n_estimators': 16},0.933333,0.943662,0.938498,0.005164


# Example 2: add multiple pipelines to example 1

In [3]:
# additional imports
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# define the model including pipelines
models = {
    'LogisticRegression': Pipeline([
        ('scale', StandardScaler()),
        ('clf', LogisticRegression(solver='lbfgs', max_iter=200))
    ]),
    'RandomForestClassifier': Pipeline([
        ('scale', StandardScaler()),
        ('clf', RandomForestClassifier())
    ])
}
params = { 
    'LogisticRegression': { 'clf__C': [0.1, 1, 2] },
    'RandomForestClassifier': { 'clf__n_estimators': [16, 32] }
}

X_train, X_test, y_train, y_test = train_test_split(X, y)
search = HyperclassifierSearch(models, params)
best_model = search.train_model(X, y, cv=10, iid=False)
search.evaluate_model()

Search for LogisticRegression ...
Search for RandomForestClassifier ...
Search is done.
results round for: LogisticRegression
results round for: RandomForestClassifier


,Estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score
0,LogisticRegression,{'clf__C': 1},0.982759,0.982759,0.982456,0.964912,0.982456,0.982456,0.947368,1.000000,1.000000,0.982143,0.980731,0.014577
1,LogisticRegression,{'clf__C': 2},0.982759,0.982759,0.982456,0.964912,0.982456,0.982456,0.947368,1.000000,1.000000,0.982143,0.980731,0.014577
2,LogisticRegression,{'clf__C': 0.1},1.000000,0.948276,0.964912,0.964912,1.000000,0.964912,0.929825,1.000000,1.000000,0.964286,0.973712,0.023788
3,RandomForestClassifier,{'clf__n_estimators': 32},0.982759,0.896552,0.947368,0.964912,1.000000,0.982456,0.964912,0.982143,1.000000,1.000000,0.972110,0.030132
4,RandomForestClassifier,{'clf__n_estimators': 16},0.982759,0.879310,0.894737,0.947368,1.000000,0.982456,0.947368,0.964286,0.964286,0.982143,0.954471,0.037327


In [4]:
# the usual parameters from GridSearchCV can be obtained:

print('highest score:', round(best_model.best_score_, 4), 
      '\n\n... with this estimator: \n\n', best_model.best_estimator_)

highest score: 0.9807 

... with this estimator: 

 Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=200,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)


# Example 3: using RandomizedSearchCV and more exhaustive search compared to example 2

In [5]:
# additional imports:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score, balanced_accuracy_score, fbeta_score

# model and parameter definition in a function
def build_model():
    models = {
        
        'LogisticRegression': Pipeline([
            ('scale', StandardScaler()),
            ('clf', LogisticRegression(solver='lbfgs'))
        ]),
        'RandomForestClassifier': Pipeline([
            ('scale', StandardScaler()),
            ('clf', RandomForestClassifier())
        ]),
        'AdaBoost': Pipeline([
            ('tfidf', StandardScaler()),
            ('clf', AdaBoostClassifier())  
        ])
    }
    params = { 
        'LogisticRegression': { 'clf__C': np.linspace(0.1, 1.0, num=10) },
        'RandomForestClassifier': { 'clf__n_estimators': np.arange(16,32+1) },
        'AdaBoost': { 'clf__n_estimators': np.arange(16,32+1) }
    }
    scorer = make_scorer(fbeta_score, beta=2, average='weighted')
    return models, params, scorer

In [6]:
# with the model defined in build_model() above the search reduces to:
models, params, scorer = build_model()
search = HyperclassifierSearch(models, params)
skf = StratifiedKFold(n_splits=5)
best_model = search.train_model(X, y, search='random', scoring=scorer, cv=skf, iid=False)
search.evaluate_model()

Search for LogisticRegression ...
Search for RandomForestClassifier ...
Search for AdaBoost ...
Search is done.
results round for: LogisticRegression
results round for: RandomForestClassifier
results round for: AdaBoost


,Estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
0,LogisticRegression,{'clf__C': 0.9},0.982609,0.982497,0.973188,0.973405,0.991139,0.980568,0.006714
1,LogisticRegression,{'clf__C': 0.7000000000000001},0.982609,0.982497,0.973188,0.973405,0.991139,0.980568,0.006714
2,LogisticRegression,{'clf__C': 0.8},0.982609,0.982497,0.973188,0.973405,0.991139,0.980568,0.006714
3,LogisticRegression,{'clf__C': 0.5},0.973869,0.973661,0.982185,0.973405,0.991139,0.978852,0.006979
4,LogisticRegression,{'clf__C': 0.30000000000000004},0.973869,0.973661,0.991122,0.964450,0.991139,0.978848,0.010590
5,LogisticRegression,{'clf__C': 1.0},0.982609,0.973869,0.973188,0.973405,0.991139,0.978842,0.007095
6,LogisticRegression,{'clf__C': 0.6},0.973869,0.982497,0.973188,0.973405,0.991139,0.978820,0.007083
7,LogisticRegression,{'clf__C': 0.4},0.973869,0.973661,0.982185,0.964450,0.991139,0.977061,0.009003
8,LogisticRegression,{'clf__C': 0.1},0.982497,0.973661,0.982185,0.964450,0.982301,0.977019,0.007125
9,LogisticRegression,{'clf__C': 0.2},0.973869,0.964766,0.982185,0.964450,0.982301,0.973514,0.007889


In [7]:
# use best model for prediction: best_model.predict(X_test)